In [41]:
import sys
from pathlib import Path
import shutil
import json
import csv
import re
import math

from shapely.geometry import Point
import requests

ROOT = Path('..')
sys.path.append(str(ROOT))

import wavetrace as wt

%load_ext autoreload
%autoreload 2

TMP_DIR = ROOT/'TMP_DIR'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# collection = {
#     'type': 'FeatureCollection', 
#     'features': list(map(wt.build_feature, tids))
#     }
# json.dumps(collection)

In [44]:
# Initialize

wt.rm_paths(TMP_DIR)
#transmitters_path = ROOT/'tests'/'data'/'transmitters.csv'
transmitters_path = ROOT/'tests'/'data'/'transmitters_single.csv'
high_definition = False
if high_definition:
    topography_path = TMP_DIR/'srtm1'
else:
    topography_path = TMP_DIR/'srtm3'


In [45]:
# Process transmitters

out_path = TMP_DIR/'splat_files'   
wt.process_transmitters(transmitters_path, out_path)

% ls {out_path}

GrayWireless_4.az  GrayWireless_4.el  GrayWireless_4.lrp  GrayWireless_4.qth


In [46]:
# Download topography

transmitters = wt.read_transmitters(transmitters_path)
print('transmitters=', transmitters)

tids = wt.get_covering_tiles_ids(transmitters)
print('Tiles to download =', tids)

wt.download_topography(tids, topography_path, 
  high_definition=high_definition)

% ls {topography_path}

transmitters= [{'antenna_downtilt': '', 'bearing': '0', 'horizontal_beamwidth': '', 'name': 'GrayWireless_4', 'power_eirp': 4.0, 'latitude': -35.712247, 'antenna_height': 10.0, 'extra': '', 'polarization': 0.0, 'network_name': 'Gray Wireless', 'frequency': 5725.0, 'longitude': 174.336547, 'site_name': '4', 'vertical_beamwidth': ''}]
Tiles to download = ['S36E173', 'S36E174', 'S37E173', 'S37E174']
S36E173.SRTMGL3.hgt.zip  S37E173.SRTMGL3.hgt.zip
S36E174.SRTMGL3.hgt.zip  S37E174.SRTMGL3.hgt.zip


In [47]:
# Process topography

out_path = TMP_DIR/'splat_files'
wt.process_topography(topography_path, out_path, 
  high_definition=high_definition)

% ll -h {out_path}

total 14M
-rw-r--r-- 1 araichev 3.5M May  8 11:44 -36:-35:185:186.sdf
-rw-r--r-- 1 araichev 4.1M May  8 11:44 -36:-35:186:187.sdf
-rw-r--r-- 1 araichev 3.6M May  8 11:44 -37:-36:185:186.sdf
-rw-r--r-- 1 araichev 2.8M May  8 11:44 -37:-36:186:187.sdf
-rw-r--r-- 1 araichev    4 May  8 11:43 GrayWireless_4.az
-rw-r--r-- 1 araichev    4 May  8 11:43 GrayWireless_4.el
-rw-r--r-- 1 araichev  335 May  8 11:43 GrayWireless_4.lrp
-rw-r--r-- 1 araichev   43 May  8 11:43 GrayWireless_4.qth


In [48]:
# Compute coverage

in_path = TMP_DIR/'splat_files'
out_path = TMP_DIR/'coverage_reports'
wt.compute_coverage(in_path, out_path, high_definition=high_definition, make_shp=True)

% ls {out_path}

GrayWireless_4-ck.png  GrayWireless_4.png  GrayWireless_4.shx
GrayWireless_4.dbf     GrayWireless_4.prj  GrayWireless_4-site_report.txt
GrayWireless_4.kml     GrayWireless_4.shp  GrayWireless_4.tif


In [49]:
# Compute satellite line-of-sight for the Optus D2 geostationary satellite 
# at 152 degrees longitude

in_path = (TMP_DIR/'srtm3'/'S36E173.SRTMGL3.hgt.zip').resolve()
satellite_lon = 152
tile_id = wt.get_tile_id(in_path)
out_path = TMP_DIR/'{!s}_satellite_los.tif'.format(tile_id)
wt.compute_satellite_los(in_path, satellite_lon, out_path)

In [50]:
# Clean up

wt.rm_paths(TMP_DIR)